<a href="https://colab.research.google.com/github/akbism/COVID-QA/blob/main/AnswerVerification/9_External_Answer_Verification_Model_Training_%26_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script is for external answer verification mechanism. It trains a logistics regression model and predicts the "No Answer" probability and class.

#Setting up the google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/Colab\ Notebooks/LJMU/covidqa/experiments

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/LJMU/covidqa/experiments


In [ ]:
class Args:
    model_type = 'bert'
    model_name = 'bert-base-cased' #'dmis-lab/biobert-base-cased-v1.1-squad'
    tokenizer_name='bert-base-cased'
    output_dir = './result'

    data_dir = '/content/gdrive/My Drive/Colab Notebooks/LJMU/covidqa/biobert-pytorch/datasets/QA/SQuAD'
    train_file = 'train-v2.0.json'
    predict_file ='dev-v2.0.json'
    
    data_dir1 = '/content/gdrive/MyDrive/Colab Notebooks/LJMU/covidqa/biobert-pytorch/datasets/QA/BioASQ/BioASQ-678b'
    train_file1 = 'BioASQ-train-factoid-7_8b-full-annotated.json'
    predict_file1 ='BioASQ-train-factoid-6b-full-annotated.json'

    data_dir2 = '/content/gdrive/MyDrive/Colab Notebooks/LJMU/covidqa/biobert-pytorch/datasets/QA'
    predict_file2 = 'COVID-QA-Modified.json'

    train_path = "./datasets/train/"
    dev_path = "./datasets/dev/"
    val_path = "./datasets/val/"

    save_model_path = "./qa_model/"

args=Args()

# metric = load_metric("squad_v2" if args.squad_v2 else "squad")

In [ ]:
!pip install pycaret[full]

     |████████████████████████████████| 302 kB 13.2 MB/s 
     |████████████████████████████████| 166.7 MB 11 kB/s 
     |████████████████████████████████| 101 kB 8.0 MB/s 
     |████████████████████████████████| 40 kB 4.1 MB/s 
     |████████████████████████████████| 51.5 MB 53 kB/s 
     |████████████████████████████████| 131 kB 71.5 MB/s 
     |████████████████████████████████| 356 kB 63.7 MB/s 
     |████████████████████████████████| 67.4 MB 65 kB/s 
     |████████████████████████████████| 345 kB 80.2 MB/s 
     |████████████████████████████████| 6.5 MB 58.6 MB/s 
     |████████████████████████████████| 904 kB 40.4 MB/s 
     |████████████████████████████████| 122 kB 71.7 MB/s 
     |████████████████████████████████| 786 kB 75.2 MB/s 
     |████████████████████████████████| 3.6 MB 39.1 MB/s 
     |████████████████████████████████| 1.8 MB 45.1 MB/s 
     |████████████████████████████████| 1.1 MB 59.7 MB/s 
     |████████████████████████████████| 5.8 MB 70.0 MB/s 
     |█████████████

In [ ]:
!mkdir encoder
!curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
!curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl
!curl -Lo ./models.py https://raw.githubusercontent.com/facebookresearch/InferSent/master/models.py
!mkdir fastText
!curl -Lo fastText/crawl-300d-2M.vec.zip https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
!unzip fastText/crawl-300d-2M.vec.zip -d fastText/

Archive:  fastText/crawl-300d-2M.vec.zip
  inflating: fastText/crawl-300d-2M.vec  


In [ ]:
import nltk
print(nltk.__version__)

3.2.5


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
import json, os
with open(os.path.join(args.data_dir, args.predict_file)) as datafile:
  data = json.load(datafile)

In [ ]:
import torch
from models import InferSent
V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [ ]:
W2V_PATH = 'fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)
infersent.build_vocab_k_words(K=100000)

Vocab size : 100000


In [ ]:
from tqdm import tqdm, trange
import json, os
from collections import defaultdict
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

training_json = {}
development_json = {}
test_json = {}
training_features = {}
training_labels = {}
feats = ["PERSON", "NORP", "FAC", "ORG", "GPE", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART", "LAW", "LANGUAGE", "DATE", "TIME", "PERCENT", "MONEY", "QUANTITY", "ORDINAL", "CARDINAL", "what", "when", "how", "many", "why", "who", 'percentage NNP', 'percentage NNPS', 'percentage CD']


In [ ]:

training_dict = {}
development_dict = {}
test_dict = {}

# ####TRAINING PREPROCESSING - Creating feature vectors
# with open(os.path.join(args.data_dir, args.train_file),'r') as f:
#     training_json = json.load(f)

# for x in tqdm(training_json['data']):
#     for p in x['paragraphs']:
#         # import pdb
#         # pdb.set_trace()
#         context = p['context']
#         doc = nlp(context)
#         labels = [x.label_ for x in doc.ents]
#         features = defaultdict(int)
#         features = Counter(labels)
#         questions = {}
#         for q in p['qas']:

#             qid = q['id']
#             questions[qid] = {q['question']:q['is_impossible']}
#             words = nltk.word_tokenize(q['question'])
#             features['what'] = int(any((True for word in words if word.lower() == "what")))
#             features['when'] = int(any((True for word in words if word.lower() == "when")))
#             features['where'] = int(any((True for word in words if word.lower() == "where")))
#             features['how'] = int(any((True for word in words if word.lower() == "how")))
#             features['many'] = int(any((True for word in words if word.lower() == "many")))
#             features['why'] = int(any((True for word in words if word.lower() == "why")))
#             features['who'] = int(any((True for word in words if word.lower() == "who")))
            
#             tagged = nltk.pos_tag(words)
#             percentPronouns = 0
#             percentNouns = 0
#             percentNumbers = 0
#             #print(tagged)
#             properNouns = [word for word,pos in tagged if pos == 'NNP']
#             numPN = len(properNouns)
#             countPN = 0
#             if numPN > 0: 
#                 for pn in properNouns:
#                     if pn in context:   
#                         countPN = countPN + 1
#                 percentPronouns = float(countPN)/numPN
            
#             nouns = [word for word,pos in tagged if pos == 'NN']
#             numNN = len(nouns)
#             countNN = 0
#             if numNN > 0:
#                 for nn in nouns:
#                     if nn in context:
#                         countNN = countNN + 1
#                 percentNouns = float(countNN)/numNN
               
#             numbers = [word for word,pos in tagged if pos == 'CD']
#             numNumbers = len(numbers)
#             countNumbers = 0
#             if numNumbers > 0:
#                 for num in numbers: 
#                     if num in context:
#                         countNumbers = countNumbers + 1
#                 percentNumbers = float(countNumbers)/numNumbers
            
#             features['percentage NNP'] = percentPronouns
#             features['percentage NN'] = percentNouns
#             features['percentage CD'] = percentNumbers
            
#             feature_vector = []
#             for i in feats:
#                 feature_vector.append(features[i])
#             training_features[qid] = feature_vector
#             training_labels[qid] = int(q['is_impossible'])
            
#         training_dict[context] = questions
        
# with open('training_features.json', 'w') as fp:
#     json.dump(training_features, fp)
# with open('training_labels.json', 'w') as fp:
#     json.dump(training_labels, fp)

In [ ]:
####DEVELOPMENT PREPROCESSING
development_features = {}
development_labels = {}

with open(os.path.join(args.data_dir, args.predict_file),'r') as f:
    development_json = json.load(f)

for x in tqdm(development_json['data']):
    for p in x['paragraphs']:
        context = p['context']
        doc = nlp(context)
        labels = [x.label_ for x in doc.ents]
        features = defaultdict(int)
        features = Counter(labels)
        questions = {}
        for q in p['qas']:
            qid = q['id']
            questions[qid] = {q['question']:q['is_impossible']}
            words = nltk.word_tokenize(q['question'])
            features['what'] = int(any((True for word in words if word.lower() == "what")))
            features['when'] = int(any((True for word in words if word.lower() == "when")))
            features['where'] = int(any((True for word in words if word.lower() == "where")))
            features['how'] = int(any((True for word in words if word.lower() == "how")))
            features['many'] = int(any((True for word in words if word.lower() == "many")))
            features['why'] = int(any((True for word in words if word.lower() == "why")))
            features['who'] = int(any((True for word in words if word.lower() == "who")))
            
            tagged = nltk.pos_tag(words)
            percentPronouns = 0
            percentNouns = 0
            percentNumbers = 0
            #print(tagged)
            properNouns = [word for word,pos in tagged if pos == 'NNP']
            numPN = len(properNouns)
            countPN = 0
            if numPN > 0: 
                for pn in properNouns:
                    if pn in context:   
                        countPN = countPN + 1
                percentPronouns = float(countPN)/numPN
            
            nouns = [word for word,pos in tagged if pos == 'NN']
            numNN = len(nouns)
            countNN = 0
            if numNN > 0:
                for nn in nouns:
                    if nn in context:
                        countNN = countNN + 1
                percentNouns = float(countNN)/numNN
               
            numbers = [word for word,pos in tagged if pos == 'CD']
            numNumbers = len(numbers)
            countNumbers = 0
            if numNumbers > 0:
                for num in numbers: 
                    if num in context:
                        countNumbers = countNumbers + 1
                percentNumbers = float(countNumbers)/numNumbers
            
            features['percentage NNP'] = percentPronouns
            features['percentage NN'] = percentNouns
            features['percentage CD'] = percentNumbers
            
            feature_vector = []
            for i in feats:
                feature_vector.append(features[i])
            development_features[qid] = feature_vector
            development_labels[qid] = int(q['is_impossible'])            
        development_dict[context] = questions


with open('development_features.json', 'w') as fp:
    json.dump(development_features, fp)
with open('development_labels.json', 'w') as fp:
    json.dump(development_labels, fp)


100%|██████████| 35/35 [01:14<00:00,  2.11s/it]


In [ ]:
# ####TESTING
# with open(os.path.join(args.data_dir2, args.predict_file2),'r') as f:
#     test_json = json.load(f)

# for x in tqdm(test_json['data']):
#     for p in x['paragraphs']:
#         context = p['context']
#         doc = nlp(context)
#         labels = [x.label_ for x in doc.ents]
#         features = defaultdict(int)
#         features = Counter(labels)
#         questions = {}
#         for q in p['qas']:
#             qid = q['id']
#             questions[qid] = q['question']
#             words = nltk.word_tokenize(q['question'])
#             features['what'] = int(any((True for word in words if word.lower() == "what")))
#             features['when'] = int(any((True for word in words if word.lower() == "when")))
#             features['where'] = int(any((True for word in words if word.lower() == "where")))
#             features['how'] = int(any((True for word in words if word.lower() == "how")))
#             features['many'] = int(any((True for word in words if word.lower() == "many")))
#             features['why'] = int(any((True for word in words if word.lower() == "why")))
#             features['who'] = int(any((True for word in words if word.lower() == "who")))
            
#             tagged = nltk.pos_tag(words)
#             percentPronouns = 0
#             percentNouns = 0
#             percentNumbers = 0
#             #print(tagged)
#             properNouns = [word for word,pos in tagged if pos == 'NNP']
#             numPN = len(properNouns)
#             countPN = 0
#             if numPN > 0: 
#                 for pn in properNouns:
#                     if pn in context:   
#                         countPN = countPN + 1
#                 percentPronouns = float(countPN)/numPN
            
#             nouns = [word for word,pos in tagged if pos == 'NN']
#             numNN = len(nouns)
#             countNN = 0
#             if numNN > 0:
#                 for nn in nouns:
#                     if nn in context:
#                         countNN = countNN + 1
#                 percentNouns = float(countNN)/numNN
               
#             numbers = [word for word,pos in tagged if pos == 'CD']
#             numNumbers = len(numbers)
#             countNumbers = 0
#             if numNumbers > 0:
#                 for num in numbers: 
#                     if num in context:
#                         countNumbers = countNumbers + 1
#                 percentNumbers = float(countNumbers)/numNumbers
            
#             features['percentage NNP'] = percentPronouns
#             features['percentage NN'] = percentNouns
#             features['percentage CD'] = percentNumbers
            
#             feature_vector = []
#             for i in feats:
#                 feature_vector.append(features[i])
#             test_features[qid] = feature_vector
#         test_dict[context] = questions

# with open('test_features.json', 'w') as fp:
#     json.dump(test_features, fp)

100%|██████████| 147/147 [03:49<00:00,  1.56s/it]


In [ ]:
## Finding the root
from nltk import Tree
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()

import json

import en_core_web_sm
nlp = en_core_web_sm.load()

# with open(os.path.join(args.data_dir, args.train_file),'r') as f:
#     training_json = json.load(f)

# question_root_dict = {} #dictionary for boolean of whether the root of the sentences is in the context 
# for x in tqdm(training_json['data']):
#     for p in x['paragraphs']:
#         sentences = []
#         context = p['context']
#         sentence = nlp(context.lower()).sents
#         for sent in sentence:
#           sentences.append(sent)
#         for q in p['qas']:
#           qid = q['id']
#           question = q['question']
#           question_root_dict[qid] = 0
#           question = question.lower()
#           qroot = st.stem(str([sent.root for sent in nlp(question).sents][0])) #if the question has 2 parts, just pick the first and stem it
#           for sent in sentences:
#             sroots = [st.stem(chunk.root.head.text.lower()) for chunk in sent.noun_chunks] 
#             if qroot in sroots:
#               question_root_dict[qid] = 1
#               break
            
# with open('question_root_train.json', 'w') as fp:
#     json.dump(question_root_dict, fp)

In [ ]:
## Finding the root
from nltk import Tree
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()

import json

import en_core_web_sm
nlp = en_core_web_sm.load()

with open(os.path.join(args.data_dir, args.predict_file),'r') as f:
    training_json = json.load(f)

question_root_dict = {} #dictionary for boolean of whether the root of the sentences is in the context 
for x in tqdm(training_json['data']):
    for p in x['paragraphs']:
        sentences = []
        context = p['context']
        sentence = nlp(context.lower()).sents
        for sent in sentence:
          sentences.append(sent)
        for q in p['qas']:
          qid = q['id']
          question = q['question']
          question_root_dict[qid] = 0
          question = question.lower()
          qroot = st.stem(str([sent.root for sent in nlp(question).sents][0])) #if the question has 2 parts, just pick the first and stem it
          for sent in sentences:
            sroots = [st.stem(chunk.root.head.text.lower()) for chunk in sent.noun_chunks] 
            if qroot in sroots:
              question_root_dict[qid] = 1
              break
            
with open('question_root_dev.json', 'w') as fp:
    json.dump(question_root_dict, fp)


100%|██████████| 35/35 [03:24<00:00,  5.84s/it]


In [ ]:
# ## Finding the root
# from nltk import Tree
# from nltk.stem.lancaster import LancasterStemmer
# st = LancasterStemmer()

# import json

# import en_core_web_sm
# nlp = en_core_web_sm.load()

# with open(os.path.join(args.data_dir2, args.predict_file2),'r') as f:
#     test_json = json.load(f)

# question_root_dict_test = {} #dictionary for boolean of whether the root of the sentences is in the context 
# for x in tqdm(test_json['data']):
#     for p in x['paragraphs']:
#         sentences = []
#         context = p['context']
#         sentence = nlp(context.lower()).sents
#         for sent in sentence:
#           sentences.append(sent)
#         for q in p['qas']:
#           qid = q['id']
#           question = q['question']
#           question_root_dict_test[qid] = 0
#           question = question.lower()
#           qroot = st.stem(str([sent.root for sent in nlp(question).sents][0])) #if the question has 2 parts, just pick the first and stem it
#           for sent in sentences:
#             sroots = [st.stem(chunk.root.head.text.lower()) for chunk in sent.noun_chunks] 
#             if qroot in sroots:
#               question_root_dict_test[qid] = 1
#               break
            
# with open('question_root_test.json', 'w') as fp:
#     json.dump(question_root_dict_test, fp)

In [ ]:
#### Reloading features
with open('training_features.json','r') as f:
    training_json = json.load(f)
training_features = training_json

with open('test_features.json','r') as f:
    test_json = json.load(f)
test_features = test_json

with open('development_features.json','r') as f:
    development_json = json.load(f)
development_features = development_json

with open('question_root_train.json', 'r') as f:
    question_root_dict_json = json.load(f)
question_root_dict_train = question_root_dict_json

with open('question_root_test.json', 'r') as f:
    question_root_dict_json = json.load(f)
question_root_dict_test = question_root_dict_json

with open('question_root_dev.json', 'r') as f:
    question_root_dict_json = json.load(f)
question_root_dict_dev = question_root_dict_json

### Target

with open('training_labels.json', 'r') as f:
    training_labels = json.load(f)

with open('development_labels.json', 'r') as f:
    development_labels = json.load(f)

In [ ]:
print(len(training_features))
print(len(test_features))
print(len(development_features))
print(len(question_root_dict_train))
print(len(question_root_dict_test))
print(len(question_root_dict_dev))
print(len(training_labels))
print(len(development_labels))

130319
2019
11873
130319
2019
11873
130319
11873


In [ ]:
import pandas as pd
df_feat_test1=pd.DataFrame.from_dict(test_features, orient='index', columns=feats)
df_feat_test2=pd.DataFrame.from_dict(question_root_dict_test, orient='index' , columns = ['question_root'])
df_feat_test=df_feat_test1.merge(df_feat_test2,  left_index=True, right_index=True)

df_feat_train1=pd.DataFrame.from_dict(training_features, orient='index', columns=feats)
df_feat_train2=pd.DataFrame.from_dict(question_root_dict_train, orient='index' , columns = ['question_root'])
df_feat_train=df_feat_train1.merge(df_feat_train2,  left_index=True, right_index=True).merge(pd.DataFrame.from_dict(training_labels, orient='index', columns=['target']),left_index=True, right_index=True)

df_feat_dev1=pd.DataFrame.from_dict(development_features, orient='index', columns=feats)
df_feat_dev2=pd.DataFrame.from_dict(question_root_dict_dev, orient='index' , columns = ['question_root'])
df_feat_dev=df_feat_dev1.merge(df_feat_dev2,  left_index=True, right_index=True).merge(pd.DataFrame.from_dict(development_labels, orient='index', columns=['target']),left_index=True, right_index=True)

In [ ]:
# # Attempt 2 -> mixing NER counts and removing extraneous ones
# new_training_features = {}
# for key in training_features:
#     features = training_features[key]
#     new_features = []
#     new_features.append(features[0])
#     new_features.append(features[1]+features[3])
#     new_features.append(features[4]+features[5])
#     new_features.append(features[2]+features[6]+features[10])
#     new_features.append(features[11]+features[12])
#     new_features.append(features[13]+features[14]+features[15]+features[17])
#     for i in range(18,len(feats)):
#         new_features.append(features[i])
#     new_training_features[key] = new_features

In [ ]:
# Attempt 3 - only keeping the last 3 features
# new_training_features = {}
# for key in training_features:
#     features = training_features[key]
#     new_features = features[-3:]
#     new_training_features[key] = new_features

# # appending the root boolean
# for key in training_features:
#     new_training_features[key].append(question_root_dict[key])

In [ ]:
df_feat_train

,PERSON,NORP,FAC,ORG,GPE,LOC,PRODUCT,EVENT,WORK_OF_ART,LAW,LANGUAGE,DATE,TIME,PERCENT,MONEY,QUANTITY,ORDINAL,CARDINAL,what,when,how,many,why,who,percentage NNP,percentage NNPS,percentage CD,question_root,target
56be85543aeaaa14008c9063,1,1,0,1,4,0,1,1,1,0,0,3,0,0,0,0,0,3,0,1,0,0,0,0,0.000000,0,0.0,0,0
56be85543aeaaa14008c9065,1,1,0,1,4,0,1,1,1,0,0,3,0,0,0,0,0,3,1,1,0,0,0,0,0.000000,0,0.0,0,0
56be85543aeaaa14008c9066,1,1,0,1,4,0,1,1,1,0,0,3,0,0,0,0,0,3,0,1,0,0,0,0,0.666667,0,0.0,0,0
56bf6b0f3aeaaa14008c9601,1,1,0,1,4,0,1,1,1,0,0,3,0,0,0,0,0,3,1,0,0,0,0,0,0.000000,0,0.0,1,0
56bf6b0f3aeaaa14008c9602,1,1,0,1,4,0,1,1,1,0,0,3,0,0,0,0,0,3,0,0,0,0,0,0,0.000000,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5a7e070b70df9f001a875439,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.000000,0,0.0,0,1
5a7e070b70df9f001a87543a,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0,0.0,0,1
5a7e070b70df9f001a87543b,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.000000,0,0.0,1,1
5a7e070b70df9f001a87543c,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.000000,0,0.0,0,1


In [ ]:
df_feat_train.dtypes

PERSON               int64
NORP                 int64
FAC                  int64
ORG                  int64
GPE                  int64
LOC                  int64
PRODUCT              int64
EVENT                int64
WORK_OF_ART          int64
LAW                  int64
LANGUAGE             int64
DATE                 int64
TIME                 int64
PERCENT              int64
MONEY                int64
QUANTITY             int64
ORDINAL              int64
CARDINAL             int64
what                 int64
when                 int64
how                  int64
many                 int64
why                  int64
who                  int64
percentage NNP     float64
percentage NNPS      int64
percentage CD      float64
question_root        int64
target               int64
dtype: object

In [ ]:
num_feats= ['PERSON',
 'NORP',
 'FAC',
 'ORG',
 'GPE',
 'LOC',
 'PRODUCT',
 'EVENT',
 'WORK_OF_ART',
 'LAW',
 'LANGUAGE',
 'DATE',
 'TIME',
 'PERCENT',
 'MONEY',
 'QUANTITY',
 'ORDINAL',
 'CARDINAL',
 'percentage NNP',
 'percentage NNPS',
 'percentage CD']

cat_feats = [ 'what',
 'when',
 'how',
 'many',
 'why',
 'who',]

In [ ]:
df_feat_train.describe().T

,count,mean,std,min,25%,50%,75%,max
PERSON,130319.0,2.061687,3.111979,0.0,0.0,1.0,3.0,88.0
NORP,130319.0,1.454446,2.515809,0.0,0.0,0.0,2.0,33.0
FAC,130319.0,0.251606,0.807768,0.0,0.0,0.0,0.0,17.0
ORG,130319.0,2.508913,2.963424,0.0,0.0,2.0,4.0,36.0
GPE,130319.0,2.292260,3.145453,0.0,0.0,1.0,3.0,48.0
LOC,130319.0,0.475311,1.149662,0.0,0.0,0.0,0.0,15.0
PRODUCT,130319.0,0.149993,0.509031,0.0,0.0,0.0,0.0,10.0
EVENT,130319.0,0.152096,0.508362,0.0,0.0,0.0,0.0,9.0
WORK_OF_ART,130319.0,0.189105,0.600724,0.0,0.0,0.0,0.0,10.0
LAW,130319.0,0.075269,0.351670,0.0,0.0,0.0,0.0,5.0


In [ ]:
from pycaret import classification
# setup the environment 
classification_setup = classification.setup(data= df_feat_train, target='target', categorical_features=cat_feats, numeric_features=num_feats)

,Description,Value
0,session_id,793
1,Target,target
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(130319, 29)"
5,Missing Values,False
6,Numeric Features,21
7,Categorical Features,7
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
classification.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.6665,0.5880,0.0085,0.5923,0.0167,0.0074,0.0378,3.897
lightgbm,Light Gradient Boosting Machine,0.6664,0.6054,0.0208,0.5329,0.0400,0.0153,0.0482,0.447
lr,Logistic Regression,0.6658,0.5713,0.0040,0.5496,0.0079,0.0031,0.0223,0.725
ridge,Ridge Classifier,0.6658,0.0000,0.0029,0.5727,0.0057,0.0024,0.0205,0.053
lda,Linear Discriminant Analysis,0.6657,0.5714,0.0042,0.5407,0.0084,0.0032,0.0221,0.173
ada,Ada Boost Classifier,0.6654,0.5746,0.0093,0.4887,0.0183,0.0058,0.0260,1.082
catboost,CatBoost Classifier,0.6634,0.6160,0.0482,0.4690,0.0875,0.0268,0.0538,21.631
xgboost,Extreme Gradient Boosting,0.6623,0.6119,0.0591,0.4619,0.1047,0.0312,0.0570,32.377
svm,SVM - Linear Kernel,0.6619,0.0000,0.0124,0.2960,0.0220,0.0009,0.0039,0.552
rf,Random Forest Classifier,0.6333,0.6003,0.2825,0.4271,0.3400,0.1005,0.1046,5.980


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=793, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
classification_xgb = classification.create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6592,0.6117,0.0538,0.4249,0.0954,0.0220,0.0403
1,0.6624,0.6079,0.0528,0.4587,0.0947,0.0275,0.0527
2,0.6648,0.6123,0.0606,0.4907,0.1079,0.0371,0.0688
3,0.6631,0.6135,0.0610,0.4709,0.1080,0.0339,0.0616
4,0.6625,0.6136,0.0554,0.4617,0.0989,0.0294,0.0552
5,0.6599,0.6057,0.0642,0.4424,0.1122,0.0299,0.0517
6,0.6638,0.6162,0.0646,0.4805,0.1138,0.0376,0.0671
7,0.6603,0.6118,0.0597,0.4417,0.1051,0.0277,0.0495
8,0.6649,0.6133,0.0639,0.4924,0.1131,0.0393,0.0713
9,0.6619,0.6130,0.0547,0.4550,0.0977,0.0279,0.0523


In [ ]:
# standardizing feature vectors and gold standard labels for logistic regression
X_train = []
y_train = []
for key in training_features:
    X_train.append(training_features[key])
    y_train.append(int(not training_labels[key]))

In [ ]:
# standardizing feature vectors and gold standard labels for logistic regression
X_test = []
y_test = []
for key in development_features:
    X_test.append(development_features[key])
    y_test.append(int(not development_labels[key]))

In [ ]:
array = df_feat_train.values
X_train = array[:, 0:28]
y_train = array[:, 28]

array = df_feat_dev.values
X_dev = array[:, 0:28]
y_dev = array[:, 28]

array = df_feat_test.values
X_test = array[:, 0:28]

In [ ]:
print(X_train.shape)
print(X_dev.shape)
print(X_test.shape)

print(y_train.shape)
print(y_dev.shape)


(130319, 28)
(11873, 28)
(2019, 28)
(130319,)
(11873,)


In [ ]:
# Feature Extraction with RFE
from pandas import read_csv
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs')
rfe = RFE(model, 10)
fit = rfe.fit(X_train, y_train)
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)
df_feat_train.columns[0:28][fit.support_]

Num Features: 10
Selected Features: [False False False False False False  True False False  True False False
  True False False False False False  True  True  True False  True  True
  True False  True False]
Feature Ranking: [14 10  3  9 16 15  1  4  7  1  2 12  1 18 11 17  6  8  1  1  1  5  1  1
  1 19  1 13]


Index(['PRODUCT', 'LAW', 'TIME', 'what', 'when', 'how', 'why', 'who',
       'percentage NNP', 'percentage CD'],
      dtype='object')

In [ ]:
# # evaluate RFE for classification
# from numpy import mean
# from numpy import std
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedStratifiedKFold
# from sklearn.feature_selection import RFE
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.pipeline import Pipeline

# # create pipeline
# rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=10)
# model = LogisticRegression(solver='lbfgs')
# pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# rfe_data = pipeline.named_steps['s'].fit(X_train, y_train)
# # evaluate model
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# n_scores = cross_val_score(pipeline, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# # report performance
# print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# pipeline.named_steps['s'].get_feature_names()
# df_feat_train.columns[0:28][pipeline.named_steps['s'].support_]

Accuracy: 0.666 (0.000)


In [ ]:
sel_feat=['PRODUCT', 'LAW', 'TIME', 'what', 'when', 'how', 'why', 'who','percentage NNP', 'percentage CD' , 'target']
array = df_feat_train[sel_feat].values
X_train = array[:, 0:10]
y_train = array[:, 10]

array = df_feat_dev[sel_feat].values
X_dev = array[:, 0:10]
y_dev = array[:, 10]

array = df_feat_test[sel_feat[0:10]].values
X_test = array[:, 0:10]

In [ ]:
# Implementation of Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_std = scaler.fit_transform(X)

clf = LogisticRegression(random_state=0, solver='lbfgs')

model = clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_dev)
print(classification_report(y_dev, y_pred))

              precision    recall  f1-score   support

         0.0       0.50      1.00      0.67      5928
         1.0       0.66      0.00      0.01      5945

    accuracy                           0.50     11873
   macro avg       0.58      0.50      0.34     11873
weighted avg       0.58      0.50      0.34     11873



In [ ]:
model.predict_proba(X_train)

array([[0.62855585, 0.37144415],
       [0.54298767, 0.45701233],
       [0.67478556, 0.32521444],
       ...,
       [0.61368276, 0.38631724],
       [0.61368276, 0.38631724],
       [0.61368276, 0.38631724]])

In [ ]:
y_train

array([0., 0., 0., ..., 1., 1., 1.])

In [ ]:
y_test_prob = model.predict_proba(X_test)
y_test_pred = model.predict(X_test)

In [ ]:
y_test_prob[:, 1]

array([0.77507271, 0.77507271, 0.73677179, ..., 0.29361886, 0.33004372,
       0.33004372])

In [ ]:
y_test_pred

array([1., 1., 1., ..., 0., 0., 0.])

In [ ]:
import numpy as np
np.mean(y_test_pred)

0.5700842000990589

In [ ]:
y_test_pred

array([1., 1., 1., ..., 0., 0., 0.])

In [ ]:
df_feat_test_sel=df_feat_test[sel_feat[0:10]]
df_feat_test_sel['no_answer_prediction']= model.predict(X_test)
df_feat_test_sel['no_answer_probability']= model.predict_proba(X_test)[:, 1]

In [ ]:
df_feat_test_sel

,PRODUCT,LAW,TIME,what,when,how,why,who,percentage NNP,percentage CD,no_answer_prediction,no_answer_probability
262,16,5,1,1,0,0,0,0,1.000000,0.0,1.0,0.775073
276,16,5,1,1,0,0,0,0,1.000000,0.0,1.0,0.775073
278,16,5,1,0,0,1,0,0,1.000000,0.0,1.0,0.736772
316,16,5,1,1,0,0,0,0,0.333333,1.0,1.0,0.786696
305,16,5,1,1,0,0,0,0,0.000000,0.0,1.0,0.823901
...,...,...,...,...,...,...,...,...,...,...,...,...
5315,0,0,1,1,0,0,0,0,1.000000,0.0,0.0,0.293619
5316,0,0,1,0,1,0,0,0,1.000000,0.0,0.0,0.261690
5317,0,0,1,1,0,0,0,0,1.000000,0.0,0.0,0.293619
5318,0,0,1,1,0,0,0,0,0.000000,1.0,0.0,0.330044


In [ ]:
import pickle
with open('covid_no_ans_features_prob.pkl', 'wb') as fid:
     pickle.dump(df_feat_test_sel, fid)

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df_feat_train.groupby('target').describe().unstack(1)

                        target
PERSON           count  0         86821.000000
                        1         43498.000000
                 mean   0             2.137985
                        1             1.909398
                 std    0             3.123768
                        1             3.082708
                 min    0             0.000000
                        1             0.000000
                 25%    0             0.000000
                        1             0.000000
                 50%    0             1.000000
                        1             1.000000
                 75%    0             3.000000
                        1             3.000000
                 max    0            88.000000
                        1            88.000000
NORP             count  0         86821.000000
                        1         43498.000000
                 mean   0             1.501238
                        1             1.361051
                 std    0    